In [106]:
# Import packages

import pandas as pd 
import numpy as np 
import os

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix

import plotly.express as px

import xgboost as xgb

from joblib import dump, load

pd.set_option('display.max_columns', None)

In [107]:
prediction_season = 2023

In [108]:
# Base data frames

point_spread_base = pd.read_csv('../pipeline_season_testing/data/final_predictive_dataframe.csv').dropna()
over_under_base = pd.read_csv('../pipeline_season_testing/data/final_predictive_dataframe.csv').dropna()
game_outcome_base = pd.read_csv('../pipeline_season_testing/data/final_predictive_dataframe.csv').dropna()
gambling_base = pd.read_csv('../pipeline_season_testing/data/final_predictive_dataframe.csv').dropna()

In [109]:
# Get relevant prediction season

point_spread_base = point_spread_base[point_spread_base.season.isin([prediction_season])]
over_under_base = over_under_base[over_under_base.season.isin([prediction_season])]
game_outcome_base = game_outcome_base[game_outcome_base.season.isin([prediction_season])]
gambling_base = gambling_base[gambling_base.season.isin([prediction_season])]

In [110]:
gambling_base

,Unnamed: 0,season,week,home,away,home_qb,away_qb,passing_value_adjusted_home,rushing_value_adjusted_home,pass_def_value_adjusted_home,rush_def_value_adjusted_home,special_teams_value_home,passing_value_adjusted_away,rushing_value_adjusted_away,pass_def_value_adjusted_away,rush_def_value_adjusted_away,special_teams_value_away,total_possession_time_standardized_home,total_possession_time_standardized_away,total_plays_standardized_home,total_plays_standardized_away,pass_percentage_standardized_home,pass_percentage_standardized_away,home_moneyline,away_moneyline,home_spread,away_spread,total_score_line,home_implied_prob,away_implied_prob
0,0,2023,2,PHI,MIN,J. Hurts,K. Cousins,0.249020,0.243856,0.154706,-0.156699,0.436085,0.497817,-0.807214,-0.322637,-0.581372,-0.412569,0.863451,-0.186567,0.501421,-0.161968,-0.607157,0.775329,-290,235,-6.5,6.5,49.0,0.743590,0.298507
1,1,2023,2,JAX,KC,T. Lawrence,P. Mahomes,-0.333177,0.139433,-0.275040,0.226625,0.346066,0.554368,0.323387,-0.192962,-0.206451,0.026861,-0.249095,-0.390414,0.064981,-0.272533,0.394374,0.258807,130,-155,3.0,-3.0,51.0,0.434783,0.607843
2,2,2023,2,BUF,LV,J. Allen,J. Garoppolo,-0.115385,0.153802,0.045670,-0.092379,-0.379250,1.466800,-0.487446,-0.230367,-0.677168,0.893819,-0.072544,-0.594261,0.181365,-1.104679,0.693692,-0.577966,-380,300,-8.5,8.5,46.5,0.791667,0.250000
3,3,2023,2,CIN,BAL,J. Burrow,L. Jackson,-0.377433,0.021855,0.215040,0.376557,-0.738406,-0.024994,0.223558,-0.020009,0.068189,0.216527,-0.475399,0.648569,-0.231798,0.105716,0.717456,-0.881952,-170,142,-3.0,3.0,46.5,0.629630,0.413223
4,4,2023,2,TEN,LAC,R. Tannehill,J. Herbert,-0.749652,0.174356,-0.212091,0.826166,0.451842,0.563022,0.360300,-0.468154,-0.302830,0.517024,-0.221219,0.334958,-0.592589,0.786562,0.335173,0.002132,136,-162,3.0,-3.0,46.0,0.423729,0.618321
5,5,2023,2,DET,SEA,J. Goff,G. Smith,0.661441,-0.221989,0.898684,-0.302952,-0.414580,-0.340669,0.170570,-0.668642,-0.302443,-0.029452,0.108654,-0.580904,0.239557,-0.278352,-0.113210,-0.132586,-238,195,-5.5,5.5,47.5,0.704142,0.338983
6,6,2023,2,TB,CHI,B. Mayfield,J. Fields,0.073503,-0.990210,-0.891161,0.589092,0.492132,-1.768333,0.026830,-0.550047,-0.613526,-0.005589,0.052320,-0.341243,0.524698,-0.464566,-0.321263,-0.277374,-148,124,-3.0,3.0,41.0,0.596774,0.446429
7,7,2023,2,ATL,GB,D. Ridder,J. Love,-0.189115,-0.077388,0.063275,0.080244,0.521615,1.882385,-1.205110,0.283696,-0.004695,0.927562,0.017862,-0.265163,-0.476205,-0.522758,-0.917788,-0.964373,105,-125,2.0,-2.0,40.5,0.487805,0.555556
9,9,2023,2,LAR,SF,M. Stafford,B. Purdy,0.608855,-0.362300,0.266151,-0.243993,-0.198270,1.026245,0.352864,0.764582,-0.222540,0.483083,0.625145,0.958115,0.297749,0.187184,-0.345997,-0.764528,275,-345,7.5,-7.5,44.0,0.266667,0.775281
10,10,2023,2,ARI,NYG,J. Dobbs,D. Jones,-0.958352,-1.543698,0.206663,0.697461,0.083051,-1.109960,0.690072,-0.295270,-0.950415,-1.232671,-0.423905,0.155696,-0.522758,-0.185245,-0.048020,-0.082004,180,-218,5.5,-5.5,40.0,0.357143,0.685535


In [111]:
# Feature lists

point_spread_feature_list = [
    'passing_value_adjusted_home'
    , 'rushing_value_adjusted_home'
    , 'pass_def_value_adjusted_home'
    , 'rush_def_value_adjusted_home'
    , 'special_teams_value_home'
    , 'passing_value_adjusted_away'
    , 'rushing_value_adjusted_away'
    , 'pass_def_value_adjusted_away'
    , 'rush_def_value_adjusted_away'
    , 'special_teams_value_away'
    , 'total_possession_time_standardized_home'
    , 'total_possession_time_standardized_away'
    , 'total_plays_standardized_home'
    , 'total_plays_standardized_away'
    , 'pass_percentage_standardized_home'
    , 'pass_percentage_standardized_away'
    , 'home_spread'
]

over_under_feature_list = [
    'passing_value_adjusted_home'
    , 'rushing_value_adjusted_home'
    , 'pass_def_value_adjusted_home'
    , 'rush_def_value_adjusted_home'
    , 'special_teams_value_home'
    , 'passing_value_adjusted_away'
    , 'rushing_value_adjusted_away'
    , 'pass_def_value_adjusted_away'
    , 'rush_def_value_adjusted_away'
    , 'special_teams_value_away'
    , 'total_possession_time_standardized_home'
    , 'total_possession_time_standardized_away'
    , 'total_plays_standardized_home'
    , 'total_plays_standardized_away'
    , 'pass_percentage_standardized_home'
    , 'pass_percentage_standardized_away'
    , 'total_score_line'
]

game_outcome_feature_list =[
    'passing_value_adjusted_home'
    , 'rushing_value_adjusted_home' 
    , 'pass_def_value_adjusted_home' 
    , 'rush_def_value_adjusted_home' 
    , 'special_teams_value_home'
    , 'passing_value_adjusted_away' 
    , 'rushing_value_adjusted_away' 
    , 'pass_def_value_adjusted_away' 
    , 'rush_def_value_adjusted_away' 
    , 'special_teams_value_away'
]

In [112]:
# Get features

point_spread_features = point_spread_base[point_spread_feature_list].to_numpy()
over_under_features = over_under_base[over_under_feature_list].to_numpy()
game_outcome_features = game_outcome_base[game_outcome_feature_list].to_numpy()

In [113]:
# Import models

point_spread_model = load('../modeling/point_spread_modeling/saved_models/ps_xg_boost_av.joblib')
over_under_model = load('../modeling/over_under_modeling/saved_models/ou_xg_boost_kinda_ok_av.joblib')
game_outcome_model = load('../modeling/game_outcome_modeling/saved_models/xg_boost_kinda_ok_av.joblib')

In [114]:
# Point Spread Predictions

home_cover_preds = point_spread_model.predict(point_spread_features)
home_cover_probs = point_spread_model.predict_proba(point_spread_features)

In [115]:
# Over Under Predictions

over_hits_preds = over_under_model.predict(over_under_features)
over_hits_probs = over_under_model.predict_proba(over_under_features)

In [116]:
# Game Winner Predictions

home_win_preds = game_outcome_model.predict(game_outcome_features)
home_win_probs = game_outcome_model.predict_proba(game_outcome_features)

In [117]:
point_spread_base['home_cover_pred'] = home_cover_preds
point_spread_base['home_cover_prob'] = home_cover_probs[:,1]

In [118]:
over_under_base['over_hits_pred'] = over_hits_preds
over_under_base['over_hits_prob'] = over_hits_probs[:,1]

In [119]:
game_outcome_base['home_win_pred'] = home_win_preds
game_outcome_base['home_win_prob'] = home_win_probs[:,1]

In [120]:
# Merge into one data frame

gambling_base = gambling_base[['season', 'week', 'home', 'away', 'home_qb', 'away_qb', 'home_spread', 'away_spread', 'total_score_line', 'home_implied_prob', 'away_implied_prob', 'home_moneyline', 'away_moneyline']]
gambling_base = gambling_base.merge(point_spread_base[['season', 'week', 'home', 'away', 'home_qb', 'away_qb', 'home_cover_pred', 'home_cover_prob']])
gambling_base = gambling_base.merge(over_under_base[['season', 'week', 'home', 'away', 'home_qb', 'away_qb', 'over_hits_pred', 'over_hits_prob']])
gambling_base = gambling_base.merge(game_outcome_base[['season', 'week', 'home', 'away', 'home_qb', 'away_qb', 'home_win_pred', 'home_win_prob']])

In [121]:
gambling_base['home_cover_advantage'] = gambling_base.apply(lambda x: x.home_cover_prob-.5, axis=1)
gambling_base['away_cover_advantage'] = gambling_base.apply(lambda x: (1-x.home_cover_prob)-.5, axis=1)

In [122]:
gambling_base['over_advantage'] = gambling_base.apply(lambda x: x.over_hits_prob-.5, axis=1)
gambling_base['under_advantage'] = gambling_base.apply(lambda x: (1-x.over_hits_prob)-.5, axis=1)

In [123]:
gambling_base['home_win_advantage'] = gambling_base.apply(lambda x: x.home_win_prob-x.home_implied_prob, axis=1)
gambling_base['away_win_advantage'] = gambling_base.apply(lambda x: (1-x.home_win_prob)-x.away_implied_prob, axis=1)

In [124]:
gambling_base

,season,week,home,away,home_qb,away_qb,home_spread,away_spread,total_score_line,home_implied_prob,away_implied_prob,home_moneyline,away_moneyline,home_cover_pred,home_cover_prob,over_hits_pred,over_hits_prob,home_win_pred,home_win_prob,home_cover_advantage,away_cover_advantage,over_advantage,under_advantage,home_win_advantage,away_win_advantage
0,2023,2,PHI,MIN,J. Hurts,K. Cousins,-6.5,6.5,49.0,0.743590,0.298507,-290,235,0,0.215485,1,0.516398,1,0.713837,-0.284515,0.284515,0.016398,-0.016398,-0.029753,-0.012344
1,2023,2,JAX,KC,T. Lawrence,P. Mahomes,3.0,-3.0,51.0,0.434783,0.607843,130,-155,1,0.677163,0,0.437360,0,0.370813,0.177163,-0.177163,-0.062640,0.062640,-0.063970,0.021344
2,2023,2,BUF,LV,J. Allen,J. Garoppolo,-8.5,8.5,46.5,0.791667,0.250000,-380,300,1,0.900437,0,0.056803,0,0.490158,0.400437,-0.400437,-0.443197,0.443197,-0.301509,0.259842
3,2023,2,CIN,BAL,J. Burrow,L. Jackson,-3.0,3.0,46.5,0.629630,0.413223,-170,142,1,0.509846,0,0.484252,0,0.110421,0.009846,-0.009846,-0.015748,0.015748,-0.519209,0.476356
4,2023,2,TEN,LAC,R. Tannehill,J. Herbert,3.0,-3.0,46.0,0.423729,0.618321,136,-162,1,0.941971,0,0.349720,1,0.551639,0.441971,-0.441971,-0.150280,0.150280,0.127911,-0.169960
5,2023,2,DET,SEA,J. Goff,G. Smith,-5.5,5.5,47.5,0.704142,0.338983,-238,195,0,0.097923,0,0.352648,1,0.886222,-0.402077,0.402077,-0.147352,0.147352,0.182080,-0.225206
6,2023,2,TB,CHI,B. Mayfield,J. Fields,-3.0,3.0,41.0,0.596774,0.446429,-148,124,1,0.679605,0,0.074611,1,0.693311,0.179605,-0.179605,-0.425389,0.425389,0.096537,-0.139739
7,2023,2,ATL,GB,D. Ridder,J. Love,2.0,-2.0,40.5,0.487805,0.555556,105,-125,1,0.606872,0,0.418600,1,0.656841,0.106872,-0.106872,-0.081400,0.081400,0.169037,-0.212397
8,2023,2,LAR,SF,M. Stafford,B. Purdy,7.5,-7.5,44.0,0.266667,0.775281,275,-345,0,0.162591,1,0.635647,0,0.123775,-0.337409,0.337409,0.135647,-0.135647,-0.142891,0.100944
9,2023,2,ARI,NYG,J. Dobbs,D. Jones,5.5,-5.5,40.0,0.357143,0.685535,180,-218,1,0.741888,1,0.572015,0,0.247406,0.241888,-0.241888,0.072015,-0.072015,-0.109737,0.067060


In [125]:
def get_point_spread_bet(game, threshold):
    if game.home_cover_advantage > game.away_cover_advantage:
        bet_team = 'home'
        bet_advantage = game.home_cover_advantage
    else:
        bet_team = 'away'
        bet_advantage = game.away_cover_advantage
        
    if bet_advantage < threshold:
        return 'none'
    
    else:
        return bet_team
    
def get_over_under_bet(game, threshold):
    if game.over_advantage > game.under_advantage:
        bet_team = 'over'
        bet_advantage = game.over_advantage
    else:
        bet_team = 'under'
        bet_advantage = game.under_advantage
        
    if bet_advantage < threshold:
        return 'none'
    
    else:
        return bet_team
    
def get_game_outcome_bet(game, threshold):
    if game.home_win_advantage > game.away_win_advantage:
        bet_team = 'home'
        bet_advantage = game.home_win_advantage
    else:
        bet_team = 'away'
        bet_advantage = game.away_win_advantage
        
    if bet_advantage < threshold:
        return 'none'
    
    else:
        return bet_team

In [126]:
gambling_final = gambling_base.copy()
gambling_final['point_spread_algo_bet'] = gambling_final.apply(lambda x: get_point_spread_bet(x, .4), axis=1)
gambling_final['over_under_algo_bet'] = gambling_final.apply(lambda x: get_over_under_bet(x, .07), axis=1)
gambling_final['game_outcome_algo_bet'] = gambling_final.apply(lambda x: get_game_outcome_bet(x, .15), axis=1)

In [127]:
def get_print_bet(game, bet_type='spread'):
    if bet_type == 'spread' and game.point_spread_algo_bet != 'none':
        return '{}, {}'.format(game[game.point_spread_algo_bet], game['{}_spread'.format(game.point_spread_algo_bet)])
    if bet_type == 'over_under' and game.over_under_algo_bet != 'none':
        return '{}'.format(game.over_under_algo_bet.capitalize())
    if bet_type == 'outcome' and game.game_outcome_algo_bet != 'none':
        return '{}, {}'.format(game[game.game_outcome_algo_bet], game['{}_moneyline'.format(game.game_outcome_algo_bet)])
    else:
        return None

In [128]:
gambling_final

,season,week,home,away,home_qb,away_qb,home_spread,away_spread,total_score_line,home_implied_prob,away_implied_prob,home_moneyline,away_moneyline,home_cover_pred,home_cover_prob,over_hits_pred,over_hits_prob,home_win_pred,home_win_prob,home_cover_advantage,away_cover_advantage,over_advantage,under_advantage,home_win_advantage,away_win_advantage,point_spread_algo_bet,over_under_algo_bet,game_outcome_algo_bet
0,2023,2,PHI,MIN,J. Hurts,K. Cousins,-6.5,6.5,49.0,0.743590,0.298507,-290,235,0,0.215485,1,0.516398,1,0.713837,-0.284515,0.284515,0.016398,-0.016398,-0.029753,-0.012344,none,none,none
1,2023,2,JAX,KC,T. Lawrence,P. Mahomes,3.0,-3.0,51.0,0.434783,0.607843,130,-155,1,0.677163,0,0.437360,0,0.370813,0.177163,-0.177163,-0.062640,0.062640,-0.063970,0.021344,none,none,none
2,2023,2,BUF,LV,J. Allen,J. Garoppolo,-8.5,8.5,46.5,0.791667,0.250000,-380,300,1,0.900437,0,0.056803,0,0.490158,0.400437,-0.400437,-0.443197,0.443197,-0.301509,0.259842,home,under,away
3,2023,2,CIN,BAL,J. Burrow,L. Jackson,-3.0,3.0,46.5,0.629630,0.413223,-170,142,1,0.509846,0,0.484252,0,0.110421,0.009846,-0.009846,-0.015748,0.015748,-0.519209,0.476356,none,none,away
4,2023,2,TEN,LAC,R. Tannehill,J. Herbert,3.0,-3.0,46.0,0.423729,0.618321,136,-162,1,0.941971,0,0.349720,1,0.551639,0.441971,-0.441971,-0.150280,0.150280,0.127911,-0.169960,home,under,none
5,2023,2,DET,SEA,J. Goff,G. Smith,-5.5,5.5,47.5,0.704142,0.338983,-238,195,0,0.097923,0,0.352648,1,0.886222,-0.402077,0.402077,-0.147352,0.147352,0.182080,-0.225206,away,under,home
6,2023,2,TB,CHI,B. Mayfield,J. Fields,-3.0,3.0,41.0,0.596774,0.446429,-148,124,1,0.679605,0,0.074611,1,0.693311,0.179605,-0.179605,-0.425389,0.425389,0.096537,-0.139739,none,under,none
7,2023,2,ATL,GB,D. Ridder,J. Love,2.0,-2.0,40.5,0.487805,0.555556,105,-125,1,0.606872,0,0.418600,1,0.656841,0.106872,-0.106872,-0.081400,0.081400,0.169037,-0.212397,none,under,home
8,2023,2,LAR,SF,M. Stafford,B. Purdy,7.5,-7.5,44.0,0.266667,0.775281,275,-345,0,0.162591,1,0.635647,0,0.123775,-0.337409,0.337409,0.135647,-0.135647,-0.142891,0.100944,none,over,none
9,2023,2,ARI,NYG,J. Dobbs,D. Jones,5.5,-5.5,40.0,0.357143,0.685535,180,-218,1,0.741888,1,0.572015,0,0.247406,0.241888,-0.241888,0.072015,-0.072015,-0.109737,0.067060,none,over,none


In [130]:
check_week = 2
gambling_final_week_check = gambling_final[gambling_final.week == check_week]

for index, game in gambling_final_week_check.iterrows():
    print('{} @ {}'.format(game.away, game.home))
    print()
    if get_print_bet(game, 'spread'):
        print(get_print_bet(game, 'spread'))
    if get_print_bet(game, 'outcome'):
        print(get_print_bet(game, 'outcome'))
    if get_print_bet(game, 'over_under'):
        print(get_print_bet(game, 'over_under'))
    print()

MIN @ PHI


KC @ JAX


LV @ BUF

BUF, -8.5
LV, 300
Under

BAL @ CIN

BAL, 142

LAC @ TEN

TEN, 3.0
Under

SEA @ DET

SEA, 5.5
DET, -238
Under

CHI @ TB

Under

GB @ ATL

ATL, 105
Under

SF @ LAR

Over

NYG @ ARI

Over

NYJ @ DAL

Under

WAS @ DEN

WAS, 145
Over

MIA @ NE

Under

NO @ CAR

CAR, 3.5
Over

CLE @ PIT

PIT, 105
Over

